<a href="https://colab.research.google.com/github/IntrovertInDisguise/iitgn-robotics/blob/main/Vignesh_Ramakrishnan_Assignment5_ITR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Q1 : Viewed UMichigan control tutorial.

In [3]:
#importing some useful libraries from miniproject
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate 
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from matplotlib import rc
from IPython.display import HTML
from numpy import sin, cos,arctan,arccos
from collections import deque
import sympy as sp
#Acceleration due to gravity
g=9.81

### Q2 : Stanford Manipulator

In [4]:
#specs
l1=10
l2=10

m1=10
m2=10
m3=20

i1= m1*(l1**2)/12
i2= m2*(l2**2)/12



In [28]:
#Reference http://www.cs.columbia.edu/~allen/F15/NOTES/stanfordinvkin.pdf
def StanfordInverseKinematics(P0,l1,l2):
    x = P0[0]
    y = P0[1]
    z = P0[2]
    phi = np.arctan2(y,x)
    r = np.sqrt((x**2) + (y**2))
    s = (z - l1)
    t = np.sqrt(((r**2) - (l2**2)))
    theta1 = phi - np.arctan2(l2,t)
    theta2 = np.arctan2(np.cos(theta1)*x + np.sin(theta1)*y, s)
    d3 = np.sin(theta2)*(np.cos(theta1)*x + np.sin(theta1)*y) + np.cos(theta2)*s

    #Alternate vals
    #    theta1 = phi - np.arctan2(l2,t)
    #    theta2 = np.arctan2(np.cos(theta1)*x - np.sin(theta1)*y, s)
    #    d3 = np.sin(theta2)*(np.cos(theta1)*x - np.sin(theta1)*y) + np.cos(theta2)*s
    
    return theta1, theta2, d3
    
#For End effector position    
def StanfordForwardKinematicsEE(theta1,theta2,d3,l1,l2):
    x = -l2*np.sin(theta1) + d3*np.sin(theta2)*np.cos(theta1)
    y = l2*np.cos(theta1) + d3*np.sin(theta2)*np.sin(theta1)
    z = d3*np.cos(theta2) + l1
    return [x,y,z]

#For J1 joint position
def StanfordForwardKinematicsJ1(theta1,theta2,d3,l1,l2):
    x = 0
    y = 0
    z = l1
    return [x,y,z]
#For J2 joint position
def StanfordForwardKinematicsJ2(theta1,theta2,d3,l1,l2):
    x = -l2*np.sin(theta1) 
    y = l2*np.cos(theta1)
    z = l1
    return [x,y,z]
def StanfordJacobian(theta1,theta2,d3,l1,l2):
  J=[[0,0,1,0,0,0],[-np.cos(theta1),-np.sin(theta1),0,l1*np.sin(theta1),-l1*np.cos(theta1),0],[0,0,0,-np.sin(theta1)*np.cos(theta2),np.cos(theta1)*np.cos(theta2),-np.sin(theta2)]]
  return J

#Post discussion with Suraj
def StanfordDynamicsCalc(theta1,theta2,d3,l1,l2):
  q1, q2, q3,q1dot, q1ddot, q2dot, q2ddot, q3dot,q3ddot=sp.symbols("q1 q2 q3 q1d q1dd q2d q2dd q3d q3dd")
  dc2=(0.5*d3) + l2
  #numpy version
  Jvc1,Jvc2,Jvc3=[[0,0,0],[0,0,0],[0,0,0]],[[-0.5*l2*np.sin(theta1)*np.cos(theta2),-0.5*l2*np.cos(theta1)*np.sin(theta2),0],[0.5*l2*np.cos(theta1)*np.cos(theta2), -0.5*l2*np.sin(theta1)*np.sin(theta2), 0],[0,0.5*l2*np.cos(theta2),0]],[[-dc2*np.sin(theta1)*np.cos(theta2),-dc2*np.cos(theta1)*np.sin(theta2),0.5*np.cos(theta1)*np.cos(theta2)],[dc2*np.cos(theta1)*np.cos(theta2),-dc2*np.sin(theta1)*np.sin(theta2),0.5*np.sin(theta1)*np.cos(theta2)],[0,dc2*np.cos(theta2),0.5*np.sin(theta2)]]
  #Sympy version
  Jvc1s,Jvc2s,Jvc3s=[[0,0,0],[0,0,0],[0,0,0]],[[-0.5*l2*sp.sin(q1)*sp.cos(q2),-0.5*l2*sp.cos(q1)*sp.sin(q2),0],[0.5*l2*sp.cos(q1)*sp.cos(q2), -0.5*l2*sp.sin(q1)*sp.sin(q2), 0],[0,0.5*l2*sp.cos(q2),0]],[[-dc2*sp.sin(q1)*sp.cos(q2),-dc2*sp.cos(q1)*sp.sin(q2),0.5*sp.cos(q1)*sp.cos(q2)],[dc2*sp.cos(q1)*sp.cos(q2),-dc2*sp.sin(q1)*sp.sin(q2),0.5*sp.sin(q1)*sp.cos(q2)],[0,dc2*sp.cos(q2),0.5*sp.sin(q2)]]
  
  Icom=[[(12/4)*(i1+i2)+(i1+i2),0,0],[0,i2+(12/4)*i2,0],[0,0,0]]
  #Jvc1=np.array(Jvc1)
  #Jvc2=np.array(Jvc2)
  #Jvc3=np.array(Jvc3)
  JtJ1=(np.transpose((Jvc1))@(Jvc1))*m1
  JtJ2=(np.transpose((Jvc2))@(Jvc2))*m2
  JtJ3=(np.transpose((Jvc3))@(Jvc3))*m3
  #Dq
  D=np.add(np.add(np.add(JtJ1,JtJ2),JtJ3),Icom)
  Dqs= (np.transpose((Jvc1s))@(Jvc1s))*m1+(np.transpose((Jvc2s))@(Jvc2s))*m2 +(np.transpose((Jvc3s))@(Jvc3s))*m3
  q=[q1,q2,q3]
  #Christoffel
  C=[[[0,0,0]]*3]*3
  SS=[[[0,0,0]]*3]*3
  for k in range(0,3):
    for j in range(0,3):
      for i in range(0,3):
        C[i][j][k] = 0.5 * (sp.diff(D[k][j], q[i]) + sp.diff(D[k][i], q[j]) - sp.diff(D[i][j], q[k]))
  for k in range(0,3):
    for j in range(0,3):
      for i in range(0,3):
        SS[i][j][k]= sp.diff(Dqs[i][j], q[k])-2*C[i][j][k]
  print("SkewSymmetric")
  print(SS)
  
  V = m1*g*l1*0.5 +  m2*g*l1 + m3*g*(l1-(d3*sp.sin(q[2])))
  Phi = [0] * 3
  C_total = 0
  D_total=0
  Tau = [0] * 3
  qdot_arr = np.array([q1dot, q2dot, q3dot])
  qddot_arr = np.array([q1ddot, q2ddot, q3ddot])
  for k in range(3): #corresponds to each link
    Phi[k] = sp.diff(V, q[k])
    for j in range(3):
      D_total = D_total + D[k][j] * qddot_arr[j]
      for i in range(3):
        C_total = C_total + C[i][j][k] * qdot_arr[i] * qdot_arr[j]
      Tau[k] = D_total + C_total + Phi[k]
  
  print("Tau")    
  
  return sp.simplify(Tau[0]),sp.simplify(Tau[1]),sp.simplify(Tau[2])
print(StanfordDynamicsCalc(10,20,5,l1,l2))



def StanfordAnimate(i):
  
  return 0
  

SkewSymmetric
[[[0, -5.0*sin(q1)**2*sin(q2)*cos(q2) - 5.0*sin(q2)*cos(q1)**2*cos(q2) + 5.0*sin(q2)*cos(q2), 0], [0, -5.0*sin(q1)**2*sin(q2)*cos(q2) - 5.0*sin(q2)*cos(q1)**2*cos(q2) + 5.0*sin(q2)*cos(q2), 0], [0, -5.0*sin(q1)**2*sin(q2)*cos(q2) - 5.0*sin(q2)*cos(q1)**2*cos(q2) + 5.0*sin(q2)*cos(q2), 0]], [[0, -5.0*sin(q1)**2*sin(q2)*cos(q2) - 5.0*sin(q2)*cos(q1)**2*cos(q2) + 5.0*sin(q2)*cos(q2), 0], [0, -5.0*sin(q1)**2*sin(q2)*cos(q2) - 5.0*sin(q2)*cos(q1)**2*cos(q2) + 5.0*sin(q2)*cos(q2), 0], [0, -5.0*sin(q1)**2*sin(q2)*cos(q2) - 5.0*sin(q2)*cos(q1)**2*cos(q2) + 5.0*sin(q2)*cos(q2), 0]], [[0, -5.0*sin(q1)**2*sin(q2)*cos(q2) - 5.0*sin(q2)*cos(q1)**2*cos(q2) + 5.0*sin(q2)*cos(q2), 0], [0, -5.0*sin(q1)**2*sin(q2)*cos(q2) - 5.0*sin(q2)*cos(q1)**2*cos(q2) + 5.0*sin(q2)*cos(q2), 0], [0, -5.0*sin(q1)**2*sin(q2)*cos(q2) - 5.0*sin(q2)*cos(q1)**2*cos(q2) + 5.0*sin(q2)*cos(q2), 0]]]
Tau
(937.415189067002*q1dd + 1.53712588996954e-14*q2dd - 2.56468594162563e-16*q3dd, 937.415189067002*q1dd + 1375.0*

In [43]:
#states[q1,q2,q3,q1d,q2d,q3d]
def xdotCalcStanford(state,t,tau1,tau2):
  dstate=np.zeros_like(state)
  #angular position states
  q1=state[0]
  q2=state[1]
  q3=state[2]
  #angular velocity states
  q1dot=state[3]
  q2dot=state[4]
  q3dot=state[5]
  #we find angular acceleration as function of torques 
  q1ddot=sp.solve(StanfordDynamicsCalc(q1,q2,q3,l1,l2),q1ddot)
  q2ddot=sp.solve(StanfordDynamicsCalc(q1,q2,q3,l1,l2),q1ddot)
  


  dstate[0]=q1dot
  dstate[1]=q2dot
  dstate[2]=q3dot
  dstate[3]=q1ddot
  dstate[4]=q2ddot
  dstate[5]=q3ddot

  return dstate.tolist()

In [45]:
#Animation code from miniproject
dt=0.02
tmax=10
t=np.linspace(0,dt,tmax)

state=StanfordInverseKinematics([1,0,1],2,2)
resultangles_forcecontrol=integrate.odeint(xdotCalcStanford, state, t,args=(tau1,tau2))

fig0 = plt.figure(figsize=(5,4))
ax = fig0.add_subplot(111,autoscale_on=False, xlim=(-(l1+l2), (l1+l2)), ylim=(-(l1+l2), 1.))
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
trace, = ax.plot([], [], ',-', lw=1)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, 's', transform=ax.transAxes)
dt=0.02
framescount=1000
def frameBuild(i):
  robot_pos_x=[0,joint[0,i],joint1[0,i],endeff[0,i]]
  robot_pos_y=[0,joint[1,i],joint1[1,i],endeff[1,i]]
  robot_pos_z=[0,joint[2,i],joint1[2,i],endeff[2,i]]

  frames_x, frames_y,frames_z = deque(maxlen=framescount), deque(maxlen=framescount), deque(maxlen=framescount)
  if i == 0:
        frames_x.clear()
        frames_y.clear()
        frames_z.clear()
  frames_x.appendleft(robot_pos_x[3])
  frames_y.appendleft(robot_pos_y[3])
  frames_z.appendleft(robot_pos_z[3])

  line.set_data(robot_pos_x, robot_pos_y,robot_pos_z)
  trace.set_data(frames_x, frames_y, frames_z)
  time_text.set_text(time_template % (i*dt))
  return (line, trace, time_text)

anim_pos = animation.FuncAnimation(fig0, frameBuild, len(resultangles), interval=dt*1000, blit=True)
HTML(anim_pos.to_html5_video())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in sqrt
  if __name__ == '__main__':


NameError: ignored

### Q3 : SCARA Manipulator

In [31]:
#specs
l0=10
l1=5
l2=5

m0=10
m1=10
m2=10
m3=10 #d3

i0= m0*((l0*0.5)**2)/3
i1= m1*((l1*0.5)**2)/3
i2= m2*((l1+l2*0.5)**2)/3


In [32]:
# From Repository code
def SCARAInverseKinematics(P0,l0,l1,l2):
    x = P0[0]
    y = P0[1]
    z = P0[2]
    r = abs(((x**2)+(y**2)-(l1**2)-(l2**2))/(2*l1*l2))
    if (r**2)>1:
        print('not possible')
    theta2 = np.arctan(np.sqrt(1-r**2)/r)
    theta1 = np.arctan(y/x) - np.arctan((l2*np.sin(theta2))/(l1+l2*np.cos(theta2)))
    d3 = l0-z
    theta1 = theta1*180/np.pi
    theta2 = theta2*180/np.pi
    return theta1,theta2,d3

def SCARAForwardKinematicsEE(theta1,theta2,d3,l0,l1,l2):
    theta1= theta1*np.pi/180
    theta2=theta2*np.pi/180
    p0= [l1*np.cos(theta1)+ l2*np.cos(theta1+theta2),
                  l1*np.sin(theta1)+ l2*np.sin(theta1+theta2),
                  l0-d3]
    return p0

def SCARAForwardKinematicsBasePrismaticJoint(theta1,theta2,d3,l0,l1,l2):
    theta1= theta1*np.pi/180
    theta2=theta2*np.pi/180
    p0= [l1*np.cos(theta1)+ l2*np.cos(theta1+theta2),l1*np.sin(theta1)+ l2*np.sin(theta1+theta2),l0]
    return p0

def SCARAForwardKinematicsJ2(theta1,theta2,d3,l0,l1,l2):
    theta1= theta1*np.pi/180
    theta2=theta2*np.pi/180
    p0= [l1*np.cos(theta1),l1*np.sin(theta1),l0]
    return p0

def SCARAForwardKinematicsJ1(theta1,theta2,d3,l0,l1,l2):
    theta1= theta1*np.pi/180
    theta2=theta2*np.pi/180
    p0= [0,0,l0]
    return p0
def SCARAJacobian(theta1,theta2,d3,l0,l1,l2):
    J=[[0,0,1,0,0,0],[0,0,1,l1*(np.sin(theta1)),-l1*(np.cos(theta1)),0],[0,0,1,(l1*np.sin(theta1))+(l2*np.sin(theta1+theta2)),-(l1*np.cos(theta1))-(l2*np.cos(theta1+theta2)),0],[0,0,0,0,0,1]]
    return J

#Post discussion with Suraj
def SCARADynamicsCalc(theta1,theta2,d3,l0,l1,l2):
    q1, q2, q3,q1dot, q1ddot, q2dot, q2ddot, q3dot,q3ddot=sp.symbols("q1 q2 q3 q1d q1dd q2d q2dd q3d q3dd")
    Jvc1,Jvc2,Jvc3=[[-0.5*l1*np.sin(theta1),0,0],[0.5*l1*np.cos(theta1),0,0],[0,0,0]],[[-l1*np.sin(theta1)-0.5*l2*np.sin(theta1+theta2),-0.5*l2*np.sin(theta1+theta2),0],[l1*np.cos(theta1)+0.5*l2*np.cos(theta1+theta2), 0.5*l2*np.cos(theta1+theta2), 0],[0,0,0]],[[-l1*np.sin(theta1)-l2*np.sin(theta1+theta2),-l2*np.sin(theta1+theta2),0],[l1*np.cos(theta1)+l2*np.cos(theta1+theta2),l2*np.cos(theta1+theta2),0],[0,0,-0.5]]
    Jvc1s,Jvc2s,Jvc3s=[[-0.5*l1*sp.sin(q1),0,0],[0.5*l1*sp.cos(q1),0,0],[0,0,0]],[[-l1*sp.sin(q1)-0.5*l2*sp.sin(q1+q2),-0.5*l2*sp.sin(q1+q2),0],[l1*sp.cos(q1)+0.5*l2*sp.cos(q1+q2), 0.5*l2*sp.cos(q1+q2), 0],[0,0,0]],[[-l1*sp.sin(q1)-l2*sp.sin(q1+q2),-l2*sp.sin(q1+q2),0],[l1*sp.cos(q1)+l2*sp.cos(q1+q2),l2*sp.cos(q1+q2),0],[0,0,-0.5]]
    Icom=[[(12/4)*(i1+i2)+(i1+i2),i2+(12/4)*i2,0],[i2+(12/4)*i2,i2+(12/4)*i2,0],[0,0,0]]
    JtJ1=(np.transpose((Jvc1))@(Jvc1))*m1
    JtJ2=(np.transpose((Jvc2))@(Jvc2))*m2
    JtJ3=(np.transpose((Jvc3))@(Jvc3))*m3
    #Dq
    D=np.add(np.add(np.add(JtJ1,JtJ2),JtJ3),Icom)
    Dqs= (np.transpose((Jvc1s))@(Jvc1s))*m1+(np.transpose((Jvc2s))@(Jvc2s))*m2 +(np.transpose((Jvc3s))@(Jvc3s))*m3
    q=[q1,q2,q3]
    #Christoffel
    C=[[[0,0,0]]*3]*3
    SS=[[[0,0,0]]*3]*3
    for k in range(0,3):
      for j in range(0,3):
        for i in range(0,3):
          C[i][j][k] = 0.5 * (sp.diff(D[k][j], q[i]) + sp.diff(D[k][i], q[j]) - sp.diff(D[i][j], q[k]))
    for k in range(0,3):
      for j in range(0,3):
        for i in range(0,3):
          SS[i][j][k]= sp.diff(Dqs[i][j], q[k])-2*C[i][j][k]
    print("SkewSymmetric")
    print(SS)
    V = m0*g*l0*0.5 +  m1*g*l0 + m2* g* l0 + m3*g*(l1-(0.5*d3*sp.sin(q[2])))
    Phi = [0] * 3
    C_total = 0
    D_total=0
    Tau = [0] * 3
    qdot_arr = np.array([q1dot, q2dot, q3dot])
    qddot_arr = np.array([q1ddot, q2ddot, q3ddot])
    for k in range(3): #corresponds to each link
      Phi[k] = sp.diff(V, q[k])
      for j in range(3):
        D_total = D_total + D[k][j] * qddot_arr[j]
        for i in range(3):
          C_total = C_total + C[i][j][k] * qdot_arr[i] * qdot_arr[j]
      Tau[k] = D_total + C_total + Phi[k]

    return sp.simplify(Tau[0]),sp.simplify(Tau[1]),sp.simplify(Tau[2])
print(SCARADynamicsCalc(10,20,5,l0,l1,l2))






SkewSymmetric
[[[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]]]
(2014.39487969338*q1dd + 1215.53077318002*q2dd, 3229.9256528734*q1dd + 2278.03077318002*q2dd, 3229.9256528734*q1dd + 2278.03077318002*q2dd + 2.5*q3dd - 245.25*cos(q3))


In [ ]:
#states[q1,q2,q3,q1d,q2d,q3d]
def xdotCalcSCARA(state,t,tau1,tau2):
  dstate=np.zeros_like(state)
  #angular position states
  q1=state[0]
  q2=state[1]
  q3=state[2]
  #angular velocity states
  q1dot=state[3]
  q2dot=state[4]
  q3dot=state[5]
  #we find angular acceleration as function of torques 
  q1ddot=sp.solver(SCARADynamicsCalc(q1,q2,d3,l0,l1,l2),q1dd,implicit=true)
  


  dstate[0]=q1dot
  dstate[1]=q2dot
  dstate[2]=q3dot
  dstate[3]=q1ddot
  dstate[4]=q2ddot
  dstate[5]=q3ddot
  return dstate.tolist()

In [ ]:


fig0 = plt.figure(figsize=(5,4,5))
ax = fig0.add_subplot(111,autoscale_on=False, xlim=(-(l1+l2), (l1+l2)), ylim=(-(l1+l2), 1.))
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
trace, = ax.plot([], [], ',-', lw=1)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, 's', transform=ax.transAxes)
dt=0.02
framescount=1000
def frameBuild(i):
  robot_pos_x=[0,joint[0,i],joint1[0,i],endeff[0,i]]
  robot_pos_y=[0,joint[1,i],joint1[1,i],endeff[1,i]]
  robot_pos_z=[0,joint[2,i],joint1[2,i],endeff[2,i]]

  frames_x, frames_y,frames_z = deque(maxlen=framescount), deque(maxlen=framescount), deque(maxlen=framescount)
  if i == 0:
        frames_x.clear()
        frames_y.clear()
        frames_z.clear()
  frames_x.appendleft(robot_pos_x[3])
  frames_y.appendleft(robot_pos_y[3])
  frames_z.appendleft(robot_pos_z[3])

  line.set_data(robot_pos_x, robot_pos_y,robot_pos_z)
  trace.set_data(frames_x, frames_y, frames_z)
  time_text.set_text(time_template % (i*dt))
  return (line, trace, time_text)

anim_pos = animation.FuncAnimation(fig0, frameBuild, len(resultangles), interval=dt*1000, blit=True)
HTML(anim_pos.to_html5_video())

### Q4 : PUMA Manipulator

In [37]:
#specs
l0=10
l1=10
l2=10
l3=10

m1=10
m2=10
m3=10

i1= m1*(l1**2)/3
i2= m2*(l2**2)/3
i3= m3*(l3**2)/3


In [42]:
# From Repository code

def PUMAInverseKinematics(P0,l1,l2,l3):
    x = P0[0]
    y = P0[1]
    z = P0[2]
    theta1 = np.rad2deg(np.arctan2(y,x))
    D = ((x**2)+(y**2)+(z-l1)*(z-l1)-l2**2-l3**2)/(2*l2*l3)
    if (D**2)>1:
        print("Not possible")
    theta3 = (np.arctan2((-np.sqrt(1-D*D)),D))
    theta2 = np.rad2deg(np.arctan2(z-l1,(np.sqrt(x**2+y**2)))-np.arctan2((l3*np.sin(theta3)),(l2+l3*np.cos(theta3))))
    theta3 = np.rad2deg(theta3)
    return theta1,theta2,theta3

def forward_kinematics_puma(theta1,theta2,theta3,l1,l2,l3):
    theta1 = theta1*np.pi/180
    theta2 = theta2*np.pi/180
    theta3 = theta3*np.pi/180
    x= np.cos(theta1)* (l3*np.cos(theta2+theta3)+ l2*np.cos(theta2))
    y= np.sin(theta1) * (l3*np.cos(theta2+theta3)+ l2*np.cos(theta2))
    z= l1+ l3*np.sin(theta2+theta3) +l2*np.sin(theta2)
    return x,y,z

def jacobianPUMA(theta1,theta2,theta3,l0,l1,l2):
    J=[[-l1*np.sin(theta1)*np.cos(theta2),l1*np.cos(theta1)*np.cos(theta2),0,0,0,1],[-l1*np.cos(theta1)*np.sin(theta2),-l1*np.sin(theta1)*np.sin(theta2),0,0,0,1]]
    return J
#Post discussion with Suraj
def PUMADynamicsCalc(theta1,theta2,theta3,l1,l2,l3):
    q1, q2, q3,q1dot, q1ddot, q2dot, q2ddot, q3dot,q3ddot=sp.symbols("q1 q2 q3 q1d q1dd q2d q2dd q3d q3dd")
    l_hyp=l3*0.5*np.cos(theta3)+l2
    l_hyps=l3*0.5*sp.cos(q3)+l2
    Jvc1,Jvc2,Jvc3=[[0,0,0],[0,0,0],[0,0,0]],[[-0.5*l1*np.sin(theta1)*np.cos(theta2),-0.5*l2*np.sin(theta2)*np.cos(theta1),0],[0.5*l2*np.cos(theta1)*np.cos(theta2), -0.5*l2*np.sin(theta1)*np.sin(theta2), 0],[0,0.5*l2*np.cos(theta2),0]],[[-l_hyp*np.sin(theta1)*np.cos(theta2),-l_hyp*np.cos(theta1)*np.sin(theta2),0.5*l3*np.cos(theta1)*np.cos(theta2)*np.sin(theta3)],[l_hyp*np.cos(theta1)*np.cos(theta2),-l_hyp*np.sin(theta1)*np.sin(theta2),0.5*l3*np.sin(theta1)*np.cos(theta2)*np.sin(theta3)],[0,l2*np.cos(theta2),0.5*l3*np.cos(theta2)]]
    Jvc1s,Jvc2s,Jvc3s=[[0,0,0],[0,0,0],[0,0,0]],[[-l1*sp.sin(q1)*0.5*sp.cos(q2),-0.5*l2*sp.cos(q1)*sp.sin(q2),0],[0.5*l2*sp.cos(q1)*sp.cos(q2), -0.5*l2*sp.sin(q1)*sp.sin(q2), 0],[0,0.5*l2*sp.cos(q2),0]],[[-l_hyps*sp.sin(q1)*sp.cos(q2),-l_hyps*sp.cos(q1)*sp.sin(q2),0.5*l3*sp.cos(q1)*sp.cos(q2)*sp.sin(q3)],[l_hyps*sp.cos(q1)*sp.cos(q2),-l_hyps*sp.sin(q1)*sp.sin(q2),0.5*l3*sp.sin(q1)*sp.cos(q2)*sp.sin(q3)],[0,l2*sp.cos(q2),0.5*l3*sp.cos(q2)]]
    Icom=[[i1+i2+i3,0,0],[0,i2+i3,i3],[0,i3,i3]]
    JtJ1=(np.transpose((Jvc1))@(Jvc1))*m1
    JtJ2=(np.transpose((Jvc2))@(Jvc2))*m2
    JtJ3=(np.transpose((Jvc3))@(Jvc3))*m3
    #Dq
    D=np.add(np.add(np.add(JtJ1,JtJ2),JtJ3),Icom)
    Dqs= (np.transpose((Jvc1s))@(Jvc1s))*m1+(np.transpose((Jvc2s))@(Jvc2s))*m2 +(np.transpose((Jvc3s))@(Jvc3s))*m3
    q=[q1,q2,q3]
    #Christoffel
    C=[[[0,0,0]]*3]*3
    SS=[[[0,0,0]]*3]*3
    for k in range(0,3):
      for j in range(0,3):
        for i in range(0,3):
          C[i][j][k] = 0.5 * (sp.diff(D[k][j], q[i]) + sp.diff(D[k][i], q[j]) - sp.diff(D[i][j], q[k]))
    for k in range(0,3):
      for j in range(0,3):
        for i in range(0,3):
          SS[i][j][k]= sp.diff(Dqs[i][j], q[k])-2*C[i][j][k]
    print("SkewSymmetric")
    print(SS)
    V = (m1*g*0.5*l1)+m2*g*(l1+0.5*l2*sp.sin(q2))+m3*g*(l1+l2*sp.sin(q2)+l3*0.5*sp.sin(q3))
    Phi = [0] * 3
    C_total = 0
    D_total=0
    Tau = [0] * 3
    qdot_arr = np.array([q1dot, q2dot, q3dot])
    qddot_arr = np.array([q1ddot, q2ddot, q3ddot])
    for k in range(3): #corresponds to each link
      Phi[k] = sp.diff(V, q[k])
      for j in range(3):
        D_total = D_total + D[k][j] * qddot_arr[j]
        for i in range(3):
          C_total = C_total + C[i][j][k] * qdot_arr[i] * qdot_arr[j]
      Tau[k] = D_total + C_total + Phi[k]
    print("tau")

    return sp.simplify(Tau[0]),sp.simplify(Tau[1]),sp.simplify(Tau[2])
print(PUMADynamicsCalc(10,20,5,l0,l1,l2))




SkewSymmetric
[[[0, -500.0*sin(q1)**2*sin(q2)*sin(q3)**2*cos(q2) - 500.0*sin(q2)*sin(q3)**2*cos(q1)**2*cos(q2) - 500.0*sin(q2)*cos(q2), 500.0*sin(q1)**2*sin(q3)*cos(q2)**2*cos(q3) + 500.0*sin(q3)*cos(q1)**2*cos(q2)**2*cos(q3)], [0, -500.0*sin(q1)**2*sin(q2)*sin(q3)**2*cos(q2) - 500.0*sin(q2)*sin(q3)**2*cos(q1)**2*cos(q2) - 500.0*sin(q2)*cos(q2), 500.0*sin(q1)**2*sin(q3)*cos(q2)**2*cos(q3) + 500.0*sin(q3)*cos(q1)**2*cos(q2)**2*cos(q3)], [0, -500.0*sin(q1)**2*sin(q2)*sin(q3)**2*cos(q2) - 500.0*sin(q2)*sin(q3)**2*cos(q1)**2*cos(q2) - 500.0*sin(q2)*cos(q2), 500.0*sin(q1)**2*sin(q3)*cos(q2)**2*cos(q3) + 500.0*sin(q3)*cos(q1)**2*cos(q2)**2*cos(q3)]], [[0, -500.0*sin(q1)**2*sin(q2)*sin(q3)**2*cos(q2) - 500.0*sin(q2)*sin(q3)**2*cos(q1)**2*cos(q2) - 500.0*sin(q2)*cos(q2), 500.0*sin(q1)**2*sin(q3)*cos(q2)**2*cos(q3) + 500.0*sin(q3)*cos(q1)**2*cos(q2)**2*cos(q3)], [0, -500.0*sin(q1)**2*sin(q2)*sin(q3)**2*cos(q2) - 500.0*sin(q2)*sin(q3)**2*cos(q1)**2*cos(q2) - 500.0*sin(q2)*cos(q2), 500.0*sin(q1)*

In [ ]:
#states[q1,q2,q3,q1d,q2d,q3d]
def xdotCalcPUMA(state,t,tau1,tau2):
  dstate=np.zeros_like(state)
  #angular position states
  q1=state[0]
  q2=state[1]
  q3=state[2]
  #angular velocity states
  q1dot=state[3]
  q2dot=state[4]
  q3dot=state[5]
  #we find angular acceleration as function of torques 



  dstate[0]=q1dot
  dstate[1]=q2dot
  dstate[2]=q3dot
  dstate[3]=q1ddot
  dstate[4]=q2ddot
  dstate[5]=q3ddot
  return dstate.tolist()

In [ ]:
initPos=[0,l3+l1/2,l2+l3/2]
FinPos=[4,3,5]
def trajtrackPUMA(initPos,FinPos):
  initq=[PUMAInverseKinematics(initPos,l1,l2,l3)]
  finalq=[PUMAInverseKinematics(FinPos,l1,l2,l3)]
  currentpos=initPos
  dt=0.02
  Kd=0.1
  Kp=5
  while(np.linalg.norm(np.array(FinPos)-np.array(currentpos)))>0.1:
    






In [ ]:
fig0 = plt.figure(figsize=(5,4,5))
ax = fig0.add_subplot(111,autoscale_on=False, xlim=(-(l1+l2), (l1+l2)), ylim=(-(l1+l2), 1.))
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
trace, = ax.plot([], [], ',-', lw=1)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, 's', transform=ax.transAxes)
dt=0.02
framescount=1000
def frameBuild(i):
  robot_pos_x=[0,joint[0,i],joint1[0,i],endeff[0,i]]
  robot_pos_y=[0,joint[1,i],joint1[1,i],endeff[1,i]]
  robot_pos_z=[0,joint[2,i],joint1[2,i],endeff[2,i]]

  frames_x, frames_y,frames_z = deque(maxlen=framescount), deque(maxlen=framescount), deque(maxlen=framescount)
  if i == 0:
        frames_x.clear()
        frames_y.clear()
        frames_z.clear()
  frames_x.appendleft(robot_pos_x[3])
  frames_y.appendleft(robot_pos_y[3])
  frames_z.appendleft(robot_pos_z[3])

  line.set_data(robot_pos_x, robot_pos_y,robot_pos_z)
  trace.set_data(frames_x, frames_y, frames_z)
  time_text.set_text(time_template % (i*dt))
  return (line, trace, time_text)

anim_pos = animation.FuncAnimation(fig0, frameBuild, len(resultangles), interval=dt*1000, blit=True)
HTML(anim_pos.to_html5_video())